# Create dataset for the problem

#### Generate data 

In [1]:
from utils.helpers import list_random_with_fixed_sum
from utils.data_types import *
import random

DATA_PATH = "Data/problem_data.xlsx"

random.seed(0)


def calculate_max_possible_products():
    max_products = 0
    for m in machines:
        if m["pallets"] == 1:
            max_products += int(
                DAILY_WORKING_MINUTES / (MAX_TIME_MACHINE + TIME_OPERATOR_FOR_EACH_WORK)
            )
        else:
            max_products += (
                int(
                    (DAILY_WORKING_MINUTES - TIME_OPERATOR_FOR_EACH_WORK)
                    // (MAX_TIME_MACHINE * m["pallets"])
                )
                * m["pallets"]
            )
    max_products *= DAYS
    return max_products


machines: list[Machine] = [
    {"id": "M1", "pallets": 1},
    {"id": "M2", "pallets": 1},
    {"id": "M3", "pallets": 2},
    {"id": "M4", "pallets": 3},
]

products: list[Product] = [
    {"id": f"RP{i}", "type": "RAW"} for i in range(0, NUMBER_OF_PRODUCTS)
] + [{"id": f"FP{i}", "type": "FINAL"} for i in range(0, NUMBER_OF_PRODUCTS)]


work_types: list[WorkType] = [
    {
        "id": f"W{i}",
        "id_raw_product": f"RP{i}",
        "id_final_product": f"FP{i}",
        "time_machine_needed": random.randrange(10, MAX_TIME_MACHINE, 10),
    }
    for i in range(0, NUMBER_OF_PRODUCTS)
]

max_products = calculate_max_possible_products()
list_desired_quantity = list_random_with_fixed_sum(max_products, NUMBER_OF_PRODUCTS)

products_desired: list[ProductDesired] = [
    {
        "id_general_product": f"P{i}",
        "start_quantity": 0,
        "total_quantity": list_desired_quantity[i],
    }
    for i, w in enumerate(work_types)
]

supplier_order: list[SupplierOrder] = [
    {
        "id": f"SO{i}",
        "id_raw_product": f"RP{i}",
        "quantity_to_add": product_desired["total_quantity"]
        - product_desired["start_quantity"],
        "day": 0,
    }
    for i, product_desired in enumerate(products_desired)
]
customer_order: list[CustomerOrder] = [
    {
        "id": f"CO{i}",
        "id_final_product": f"FP{i}",
        "quantity_to_remove": product_desired["total_quantity"],
        "day": DAYS - 1,
    }
    for i, product_desired in enumerate(products_desired)
]

#### Write on excel

In [2]:
import pandas as pd

machines = pd.DataFrame(machines)
products = pd.DataFrame(products)
work_types = pd.DataFrame(work_types)
supplier_order = pd.DataFrame(supplier_order)
customer_order = pd.DataFrame(customer_order)
products_desired = pd.DataFrame(products_desired)

with pd.ExcelWriter(DATA_PATH) as writer:
    machines.to_excel(writer, sheet_name="machines", index=False)
    products.to_excel(writer, sheet_name="products", index=False)
    work_types.to_excel(writer, sheet_name="work types", index=False)
    supplier_order.to_excel(writer, sheet_name="supplier order", index=False)
    customer_order.to_excel(writer, sheet_name="customer order", index=False)
    products_desired.to_excel(writer, sheet_name="products desired", index=False)